In [9]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import time
import csv

In [14]:
square_p = 20  #green square for trimming
eps = 15 
def trim_img(img, plot = False): #try rapid trim image algo
    """
    Description: Crop the pram img to the environement space
    
    """
    ti = np.copy(img)
    # find red squares
    if plot: print("starting trimming img")
   
    square = cv2.imread('IP/image/fond.png',cv2.IMREAD_COLOR)
    #resize template
    square = cv2.resize(square,(square_p,square_p))
    
    #for debug
    if plot:
        plt.imshow(square[:,:,::-1])
        plt.show()
        
    res =  cv2.matchTemplate(img,square, cv2.TM_CCORR_NORMED)
    
    if plot:
        plt.imshow(res[:,:])
        plt.show()
      
    tresh = res[cv2.minMaxLoc(res)[3][1],cv2.minMaxLoc(res)[3][0]]
    tresh = .98*tresh
    
    find = np.where(res>tresh)

    minx = min(find[0])
    maxx = max(find[0])
    miny = min(find[1])
    maxy = max(find[1])
    ti = ti[minx-eps:maxx+eps,miny-eps:maxy+eps]    
    # security check
    if abs(maxx-minx) < 0.5*img.shape[0] or abs(maxy-miny) < 0.5*img.shape[1]:
        print("WARNING: wrong trimming very likely")
        print(f"maxx: {maxx}, minx: {minx}, maxy: {maxy}, miny: {miny}")
        ti = img
    if plot: print("Trim finished")
    return ti

In [15]:
thymio_mask_front_size = 1 #real size of the thymio mask: one dim cause circle or square 
thymio_mask_back_size = 1
def find_thymio(trim_img, plot = False):
    """
    Find thymio robot pose on an already trimmed image. The function will return a ~random value if thymio is not present on the image.
    
    param:
    -trim_img: the image trimmed, i.e. the map in image form.
    
    return: x,y,orientation
    -postion of thymio [x,y]
    -orientation of thymio: 
    """
    if plot: print("start finding thymio")
    world_size = (118.9,2*84.1)

    
    ti_shape = trim_img.shape
    ratio = (ti_shape[0]/world_size[0]+ti_shape[1]/world_size[1])/2 
    
    mask_pixel_front = (int(ratio*thymio_mask_front_size),int(ratio*thymio_mask_front_size))
    mask_pixel_back = (int(ratio*thymio_mask_back_size),int(ratio*thymio_mask_back_size))
    template_front = cv2.imread('IP/image/cb.png',cv2.IMREAD_COLOR)
    template_back = cv2.imread('IP/image/co.png',cv2.IMREAD_COLOR)
    
    #resize template
    template_front = cv2.resize(template_front,(mask_pixel_front[1],mask_pixel_front[0]))
    template_back = cv2.resize(template_back,(mask_pixel_back[1],mask_pixel_back[0]))
    
    res_front = cv2.matchTemplate(trim_img,template_front,cv2.TM_CCORR_NORMED)
    res_back = cv2.matchTemplate(trim_img,template_back,cv2.TM_CCORR_NORMED)
    
    if plot: 
        plt.imshow(res_front[:,:])
        plt.show()
        plt.imshow(res_back[:,:])
        plt.show()
    
    tresh_front = res_front[cv2.minMaxLoc(res_front)[3][1],cv2.minMaxLoc(res_front)[3][0]]
    tresh_back = res_back[cv2.minMaxLoc(res_back)[3][1],cv2.minMaxLoc(res_back)[3][0]]

    
    w = template_front.shape[1]
    h = template_front.shape[0]
    front = np.array(np.where(res_front >= tresh_front)) 
    back = np.array(np.where(res_back >= tresh_back))
   
    
    pt =  back + 2/7*(front-back)
    pt = np.array([pt[1],pt[0]]).astype(int)
    front = np.array([front[0]+w//2,front[1]+h//2])   
    back = np.array([back[0]+w//2,back[1]+h//2])
    
    pos = back + 2/7*(front-back) #find hole of the thymio, empirical: 2/7 
    #change to have origin en bas a gauche
    pos = pos.astype(int)
     
    orientation = np.arctan2((front-back)[1][0],(front-back)[0][0])
    if orientation < 0:  orientation += 2*np.pi #change to [0,2pi[
    orientation = (orientation - np.pi/2)%(2*np.pi) #change to new axis
    
    if plot:
       
        plot_img = np.copy(trim_img)
        
        coord = (pt[0][0]+w,pt[1][0]+h)
        #print(pos,coord)
        cv2.rectangle(plot_img,(pt[0][0],pt[1][0]),coord,(0,0,255),10)
        plt.imshow(plot_img[:,:,::-1])
        plt.show()
        print("finished finding thymio")
        
    x = ti_shape[0]-pos[0][0]
    y = pos[1][0]
    return y,x, orientation

In [16]:
def template_matching (img, template, approx_shape, r ,method = cv2.TM_CCOEFF_NORMED,verbose = False, plot = False, intensity = 0.6):
    """
    Perform template matching algorithm on a given image with a given template. The approximate pixels shape of the template must be given for sucessful template matching.
    
    param img: image to match template
    param template: image that defines the matching pattern to find in img
    param approx_shape: approximate shape (in pixels) of template on the img
    param r: sensisivity parameter to detect several templates; to tune
    param intensity: sensivity paremeter to detect the template (avoid detecting noise if no template is present); to tune
    param method: template matching method
    
    return img_labelled: binary image of same shape as img; 1 if this pixel contains the template, 0 otherwise
    
    """
    #resize template
    template = cv2.resize(template,(approx_shape[1],approx_shape[0]))
    #match
    
    
    res = cv2.matchTemplate(img,template,method)
    tresh = res[cv2.minMaxLoc(res)[3][1],cv2.minMaxLoc(res)[3][0]]
    #print(tresh)
    if tresh < intensity: #prevent finding from indexing noise when finding nothing
        tresh = np.inf
    tresh = tresh*r #to find multiple objects
    loc = np.where(res >= tresh)
    w = template.shape[1]
    h = template.shape[0]
    
    img_labelled = np.zeros((img.shape[0],img.shape[1]))

    if tresh == np.inf:
        print("WARNING: template not detected")
        return img_labelled 
    else:
        if plot:
            plt.imshow(res[:,:])
            plt.show()
            new_img = np.copy(img)
        S = len(loc[0])
        for pt in zip(*loc[::-1]):
            coord = (pt[0]+w,pt[1]+h)
            if plot:
                cv2.rectangle(new_img,pt,coord,(0,0,255),2)
            for i in range(pt[1],coord[1]): #draw rectangle
                for j in range(pt[0],coord[0]):
                    img_labelled[i,j] = 1


        if plot:
            plt.imshow(new_img[:,:,::-1])
            plt.show()
        return img_labelled 


In [17]:
## usefull functions: 
def defWorldSize (img):
    s = img.shape
    hor = (118.9,2*84.1) #double A0 format
    ver = (2*84.1,118.9) 
    if s[0] > s[1]:
        return ver, cv2.rotate(img,cv2.ROTATE_90_CLOCKWISE)
    else:
        return hor,img

#could optimize for several OR at the time, 
#but no need as it is already very fast with low resolution img 
def OR (matrix1, matrix2):
    m = np.zeros(matrix1.shape)
    for i in range(matrix1.shape[0]):
        for j in range(matrix1.shape[1]):
            if matrix1[i,j] or matrix2[i,j]: m[i,j]=1
    return m
    


In [1]:
def pipeline_IP(plot = False):
    """
    !! need the image folder in the same directory !!
    Description:
    runs the whole pipeline to build the map of the environement. 
    Performs global image analysis for our mobile project task: Basics of mobile robotics, MICRO-452.
    
    param plot: show graphs and information on where the program holds
    
    return: 
    1) multiple matrix with the same shape as the img param / 10 (in 2D) in the output folder:
        roads: matrix with roads labelled by 1, 0 otherwise
        passage: matrix with crossroads labelled by 1, "
        goal: "          target labelled by 1
        obst: matrix with roads labbelled by a 1, but with the location of the crossroads removed if they overlap
    2) (x,y), orientation: x, y postion of the thymio robot in the world and its orientation
    """
    
    
    start_time = time.time()
    
    img = cv2.imread('img.jpg',cv2.IMREAD_COLOR)
    world_size,img = defWorldSize(img)

    reducer = 10
    desiredMapShape = (img.shape[0]//reducer,img.shape[1]//reducer)
    
    if plot:
        plt.imshow(img[:,:,::-1])
        plt.show()
        
    ### objects real size
    road_real_size_a = (12.5,12.5)
    road_real_size_s = (6,12.5)
    passage_real_size_s = (14,14) 
    goal_real_size = (2,2) ##change goal


    ### tresholds // can go change them manually
    #square_tresh = .99
    road_angle_tresh = .999
    road_straigth_tresh = .99
    passage_tresh = .97
    
    ###trim the img
    ti = trim_img(img,plot = plot) 
    ti_pixels = ti.shape
    if plot: 
        plt.imshow(ti[:,:,::-1])
        plt.show()
    ratio = (ti_pixels[0]/world_size[0]+ti_pixels[1]/world_size[1])/2
    
    ###find thymio pose in high resolution
    x,y,orientation = find_thymio(ti,plot)
    x = int(x/ti_pixels[1]*desiredMapShape[1]) #convert to the coord of the desired output map shape
    y = int(y/ti_pixels[0]*desiredMapShape[0])
    
    ###find goal 
    if plot: print("start detecting target position")
    goal_img = cv2.imread('IP/image/goal.jpg',cv2.IMREAD_COLOR)   
    goal_pixel = (int(ratio*goal_real_size[0]),int(ratio*goal_real_size[1])) 
    #filtered_img = ti - cv2.GaussianBlur(ti, (0,0), 3) 
    matchgoal = template_matching(ti,goal_img,goal_pixel,r=1,plot = plot, verbose = plot,method = cv2.TM_CCORR_NORMED)
    #matchgoal2 = template_matching(filtered_img,cv2.rotate(goal_img,cv2.ROTATE_90_CLOCKWISE),(goal_pixel[1],goal_pixel[0]),r=1,plot = plot, verbose = plot,method = cv2.TM_CCORR_NORMED,intensity = .4)
    #matchgoal3 = template_matching(filtered_img,cv2.rotate(goal_img,cv2.ROTATE_90_COUNTERCLOCKWISE),(goal_pixel[1],goal_pixel[0]),r=1,plot = plot, verbose = plot,method = cv2.TM_CCORR_NORMED,intensity = .4)
    #matchgoal4 = template_matching(filtered_img,cv2.rotate(goal_img,cv2.ROTATE_180),goal_pixel,r=1,plot = plot, verbose = plot,method = cv2.TM_CCORR_NORMED,intensity = .4)
    #matchgoal = OR(OR(matchgoal3,matchgoal4),OR(matchgoal1,matchgoal2))
    if plot:
        plt.imshow(matchgoal[:,:])
        plt.show()
        print("finish detecting target")
    
    
    ###reduce resolution to decrease computation
    ti = cv2.resize(ti,(desiredMapShape[1],desiredMapShape[0]))
    ratio = (desiredMapShape[0]/world_size[0]+desiredMapShape[1]/world_size[1])/2 #estimate ratio: pixels/cm 
    if plot:
        plt.imshow(ti[:,:,::-1])
        plt.show()

    ###find roads 
    road_a =cv2.imread('IP/image/road_angle.png',cv2.IMREAD_COLOR)
    road_s = cv2.imread('IP/image/road_straight.png',cv2.IMREAD_COLOR)
    road_pixels_a = (int(ratio*road_real_size_a[0]),int(ratio*road_real_size_a[1]))
    road_pixels_s = (int(ratio*road_real_size_s[0]),int(ratio*road_real_size_s[1]))
    
    if plot: print("start detecting edge roads")
    match1 = template_matching(ti,road_a,road_pixels_a,road_angle_tresh, verbose = plot,method = cv2.TM_CCORR_NORMED, plot = plot)
    match2= template_matching(ti,cv2.rotate(road_a,cv2.ROTATE_90_CLOCKWISE),(road_pixels_a[1],road_pixels_a[0]),road_angle_tresh,method = cv2.TM_CCORR_NORMED, verbose = plot, plot = plot)
    match3 = template_matching(ti,cv2.rotate(road_a,cv2.ROTATE_180),road_pixels_a,road_angle_tresh, verbose = plot,method = cv2.TM_CCORR_NORMED, plot = plot)
    match4 = template_matching(ti,cv2.rotate(road_a,cv2.ROTATE_90_COUNTERCLOCKWISE),(road_pixels_a[1],road_pixels_a[0]),road_angle_tresh,verbose = plot,method = cv2.TM_CCORR_NORMED, plot = plot) 
    if plot: print("start detecting straigth roads")
    match5 = template_matching(ti,road_s,road_pixels_s,road_straigth_tresh, verbose = plot, method = cv2.TM_CCORR_NORMED,plot = plot)
    match6 = template_matching(ti,cv2.rotate(road_s,cv2.ROTATE_90_CLOCKWISE),(road_pixels_s[1],road_pixels_s[0]),road_straigth_tresh,method = cv2.TM_CCORR_NORMED, verbose = plot, plot = plot)
    
    result_r = OR(OR(OR(OR(OR(match1,match2),match3),match4),match5),match6) 
    if plot: 
        plt.imshow(result_r[:,:])
        plt.show()
        print("finish detecting roads")
    
    ###find crossroads
    if plot: print("start detecting crossroads")
    passage_img = cv2.imread('IP/image/passage.png',cv2.IMREAD_COLOR)   
    passage_pixels=(int(ratio*passage_real_size_s[0]),int(ratio*passage_real_size_s[1]))
    match7 = template_matching(ti,passage_img,passage_pixels,passage_tresh, plot = plot, verbose = plot,intensity = .1,)
    match8 = template_matching(ti,cv2.rotate(passage_img,cv2.ROTATE_90_CLOCKWISE),(passage_pixels[1],passage_pixels[0]),passage_tresh,intensity = .1, plot = plot, verbose = plot)

    result_p = OR(match7,match8)
    if plot:
        plt.imshow(result_p[:,:])
        plt.show()
        print("finish detecting cross roads")
    
    #treat cross roads and roads 
    result_o = np.zeros(result_p.shape)
    for i in range(result_p.shape[0]):
        for j in range(result_p.shape[1]):
            if result_p[i,j] and result_r[i,j]:
                result_o[i,j] = 0;
            elif result_r[i,j]:
                result_o[i,j] = 1;
    if plot:
        plt.imshow(result_o[:,:])
        plt.show()
       
    with open('output/roads.csv', 'w', encoding='UTF8', newline='') as f:
        writer = csv.writer(f)

        # write multiple rows
        writer.writerows(result_r)
    
    with open('output/passage.csv', 'w', encoding='UTF8', newline='') as f:
        writer = csv.writer(f)

        # write multiple rows
        writer.writerows(result_p)
    with open('output/obst.csv', 'w', encoding='UTF8', newline='') as f:
        writer = csv.writer(f)

        # write multiple rows
        writer.writerows(result_o)
 
    with open('output/goal.csv', 'w', encoding='UTF8', newline='') as f:
        writer = csv.writer(f)

        # write multiple rows
        writer.writerows(matchgoal)
        
    
    if plot: print("Image Processing finished in %s seconds " % (time.time() - start_time))
    
    return (x,y), orientation




In [ ]:
#pipeline_IP(True)